### Installation

In [ ]:
# !pip install --upgrade matplotlib
# !pip install --upgrade seaborn
# !pip install --ignore-installed pyyaml==5.1
# !pip install torch==1.7
# !pip install torchvision==0.8.1

# import torch, torchvision
# print(torch.__version__, torch.cuda.is_available())
# !gcc --version

# import getpass
# import os
# password = getpass.getpass()
# command = "sudo -S apt install tesseract-ocr " 
# os.system('echo %s | %s' % (password, command))

# !pip install pytesseract

# import torch, torchvision
# torch.__version__
# assert torch.__version__.startswith("1.7")

# !pip install 'git+https://github.com/facebookresearch/detectron2.git' (previously worked)
# !git clone https://github.com/facebookresearch/detectron2.git

# !pip install opencv-python

# conda install pytorch torchvision torchaudio -c pytorch

# !pip install camelot-py[plot]
# !pip install pdf2image

# !pip install ghostscript
# !pip install camelot-py[cv]
# !pip install excalibur-py
# !apt install ghostscript python3-tk
# from ctypes.util import find_library
# print(find_library("gs")) #will display libgs.so.9 if installed; will print None if not
# !excalibur initdb -> will throw ModuleNotFoundError: No module named 'camelot.ext'

# conda install -c conda-forge poppler

# !pip install pickle-mixin

In [ ]:
# !pip install --upgrade numpy

In [ ]:
# !git clone https://github.com/Psarpei/Multi-Type-TD-TSR.git
# !mv Multi-Type-TD-TSR/ Multi_Type_TD_TSR

In [ ]:
# Installing detectron2
# https://gilberttanner.com/blog/detectron-2-object-detection-with-pytorch

# Building of Facebook's detectron2
# Source: https://github.com/facebookresearch/detectron2/issues/300
# Mac OS does not support CUDA, we need to force detectron2 to depend on cpu 
# change the line to -> model.to(torch.device("cpu")) 
# Directory: Macintosh HD > opt > anaconda3 > lib > python 3.8 > site-packages > detectron2 > modelling > meta_arch > build.py

3 additional dependencies: model_final.pth, All_X152.yaml, Base-RCNN-FPN.yaml (put all in Desktop folder)
- For All_X152.yaml, access the file and change _BASE_ directory based on where you store 

### Importing Libraries 

In [ ]:
import torch, torchvision
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
import pytesseract

In [ ]:
from platform import python_version
print(python_version())

### Preparing Multi-Type-TD-TSR

In [ ]:
import detectron2
import Multi_Type_TD_TSR.google_colab.deskew as deskew
import Multi_Type_TD_TSR.google_colab.table_detection as table_detection
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_all as tsra
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_lines as tsrl
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_wol as tsrwol
import Multi_Type_TD_TSR.google_colab.table_structure_recognition_lines_wol as tsrlwol
import Multi_Type_TD_TSR.google_colab.table_xml as txml
import Multi_Type_TD_TSR.google_colab.table_ocr as tocr
import pandas as pd
import os
import json
import itertools
import random
from detectron2.utils.logger import setup_logger
# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog
setup_logger()

In [ ]:
#create detectron config
cfg = get_cfg()

#set yaml
cfg.merge_from_file('/Users/aifen/Desktop/All_X152.yaml')

#set model weights
cfg.MODEL.WEIGHTS = '/Users/aifen/Desktop/model_final.pth' # Set path model .pth

predictor = DefaultPredictor(cfg) 

In [ ]:
from PyPDF2 import PdfFileReader
import requests, io
import gzip
import requests
import pdf2image
import cv2
import camelot
import numpy as np
import pandas as pd

### Table Detection (Jermaine)

In [ ]:
# check if two rectangles overlap by using top left and bottom right coordinates
def isoverlap(r1, r2):
    y1 = r1[1]
    x1 = r1[0]
    h1 = r1[3]
    w1 = r1[2]
    
    y2 = r2[1]
    x2 = r2[0]
    h2 = r2[3]
    w2 = r2[2]
    
    if ((x1+w1)<x2 or (x2+w2)<x1 or (y1+h1)<y2 or (y2+h2)<y1):
        return False
    else:
        return True

In [ ]:
# Function: Discard overlapping tables detected
def check_coord_overlap(table_coords):
    
    new_table_coords = []
    
    for i in range (len(table_coords)):
        flag = True
        if i == 0:
            new_table_coords.append(table_coords[i])
        else:
            for j in range (len(new_table_coords)):
                if isoverlap(table_coords[i], new_table_coords[j]):
                    flag = False
                    break
            if flag:
                new_table_coords.append(table_coords[i])
    
    return new_table_coords    

In [ ]:
# list of keywords
ESG_DICTIONARY = ['ghg emissions', 'scope 1', 
                'scope 2', 'scope 3', 'energy', 
                'paper', 'green bonds', 'renewable energy',
                'water', 'carbon intensity', 
                'carbon emissions', 'waste', 
                'electricity', 
                'weighted average carbon intensity', 'WACI']

# list of units
ESG_UNITS = ['tonnes', 'tons', 'kWh', 'kilogram', 'kilowatt hour', 
           'gigajoules', 'GJ', 'litre', 'liter', 'CO2e', 'tCO', 't CO', 'MWh', 
           'megawatt hour', 'GWh', 'gigawatt hour',
           'cubic metres', 'cm3', 'm3', 'per employee', 'ream', 'quire', 'sheet', 'bundle', 'bale']


### Table Extraction (Ai Fen)

#### Extraction

In [ ]:
def get_pdf_dimension(pdf_url):
    response = requests.get(pdf_url)
    with io.BytesIO(response.content) as open_pdf_file:
        pdf = PdfFileReader(open_pdf_file, strict=False)
        height = pdf.getPage(0).mediaBox.getHeight()
        width = pdf.getPage(0).mediaBox.getWidth()
    return height, width

# https://stackoverflow.com/questions/51481200/convert-pdf-to-image-using-pdf-url-pdf2image
# https://stackoverflow.com/questions/65180516/python-pdf2image-from-link-unable-to-get-page-count **

In [ ]:
def get_image_and_dimension(pdf_url):
    response = requests.get(pdf_url, stream=True, timeout=30)
    # pdf = gzip.open(response.raw)
    # images = pdf2image.convert_from_bytes(pdf.read())
    images = pdf2image.convert_from_bytes(response.content, size=1000)
    pg_1_img = images[0] # type=PIL.PpmImagePlugin.PpmImageFile
    width, height = pg_1_img.size
    return height, width, images

In [ ]:
def get_scaling_factor(pdf_height, pdf_width, img_height, img_width):
    scaling_factor_height = img_height/pdf_height
    scaling_factor_width = img_width/pdf_width
    return scaling_factor_height, scaling_factor_width

In [ ]:
def convert_PIL_cv2(pdf_pil_img):
    pdf_cv2_img = []
    for pil_img in pdf_pil_img:
        cv2_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        pdf_cv2_img.append(cv2_img)
    return pdf_cv2_img

#### Cleaning 

In [ ]:
# cleaning up tables 
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def num_there(s):
    return any(i.isdigit() for i in s)

# list of units to be used for filtering - note: this is different from the ones under Table Detection section! 
UNITS = ['tonnes', 'tons', 'kWh', 'kilogram', 'kilowatt hour', 
           'gigajoules', 'GJ', 'litre', 'liter', 'CO2e', 'tCO', 't CO', 'MWh', 
           'megawatt hour', 'GWh', 'gigawatt hour',
           'cubic metres', 'cm3', 'm3', 'per employee', 'ream', 'quire', 'sheet', 'bundle', 'bale', '%', 't']

In [ ]:
# return cleaned tbl and list of words 
def clean_tbl(tbl):
    # replace subscript and newline 
    tbl = tbl.replace(r'(<s>).*(</s>)','',regex=True)
    tbl = tbl.replace(r'\n','',regex=True)
    # convert 1st row to header 
    header_df = tbl.iloc[0] #grab the first row for the header
    tbl = tbl[1:]
    tbl.columns = header_df 
    # remove comma in numeric values 
    tbl = tbl.apply(lambda x: x.str.replace(',',''))
    # remove brackets surrounding numeric metrics 
    tbl = tbl.replace(r"\((\d+)\)", r"\1", regex=True)
    # loop through each cell and check if they are float/num or they are metrics with units 
    for row in range(tbl.shape[0]):
        for col in range(1, tbl.shape[1]):
            value = tbl.iloc[row, col]
            if len(value.split()) > 3:
                tbl.iloc[row,col] = np.nan
            elif isfloat(value) or (any(substring in value for substring in UNITS) and num_there(value)):
                continue 
            else:
                tbl.iloc[row,col] = np.nan
    # drop columns with > 80% NaN
    tbl = tbl.loc[:, tbl.isnull().mean() < .8]
    # drop rows with any NaN
    tbl = tbl.dropna()
    print('here1')
    if (tbl.shape[1] == 1) or (tbl.shape[0] == 0): # if there's only 1 col left or 0 row left 
        return None, None 
    page_kw = ['page', 'Page', 'PAGE']
    for s in page_kw:
        if any(s in h for h in tbl.columns):
            return None, None 
    print('here2')
    first_column = tbl.iloc[:, 0] # get first column of tbl 
    num_of_nan = first_column.isnull().sum(axis = 0)
    # large proportion of nan cells in 1st column
    if num_of_nan/len(first_column) > 0.8:
        return None, None
    print('here3')
    # no headers 
    headers =tbl.columns
    if not(any(h for h in headers)):
        return None, None 
    print('here4')
    # list of words in df for relevance 
    words = pd.unique(tbl.values.ravel())
    words = pd.unique([word for line in words for word in line.split()])
    final_words = []
    print('here5')
    for s in ESG_DICTIONARY:
        if any(s in word for word in words):
            final_words.append(s) 
    print('here7')
    return tbl, final_words 

### Combined Pipeline

In [ ]:
# extract text-based and image-based tbl from individual page 
def extract_tbl_from_page(page_idx, pdf_cv2_img, scaling_factor_height, scaling_factor_width, img_height, url):
    table_list, table_coords = table_detection.make_prediction(pdf_cv2_img[page_idx], predictor)
    tbl_lst = []
    keywords_lst = []
    # img_lst = []
    
    table_coords = check_coord_overlap(table_coords) # newly added 
    
    count = -1
    # ---- start Table extraction ---- /recount from every page 
    # i = 0 
    # ---- end Table extraction ----
    for table_coord in table_coords:
        count += 1
        try: 
            x1 = table_coord[0]
            y1 = table_coord[1]
            x2 = table_coord[2] + table_coord[0]
            y2 = table_coord[3] + table_coord[1]
            
            # extract detected table by coordinates (Jermaine's part)
            page_img = pdf_cv2_img[page_idx]
            page_img = page_img[y1:y2, x1:x2]
            scale_percent = 150 # percent of original size
            width = int(page_img.shape[1] * scale_percent / 100)
            height = int(page_img.shape[0] * scale_percent / 100)
            dim = (width, height)
            # resize image
            resized = cv2.resize(page_img, dim)
            #cv2_imshow(resized)
            text = pytesseract.image_to_string(resized)
            
            # if any keyword found in text in table, save image into output folder
            if any(keyword in text.lower() for keyword in ESG_DICTIONARY) and any(unit in text for unit in ESG_UNITS):
                print("----------------------------------------------------------------------------------------------------------")
                print("RELEVANT IMAGE FOUND!, tbl_num is {0}".format(count))
                print("----------------------------------------------------------------------------------------------------------")
                
                # cv2 and pdf used 2 different coordinate systems, thus y1 and y2 need to be modified
                y1 = img_height - y1
                y2 = img_height - y2
                
                # scale by scaling factors
                scaled_x1 = x1/scaling_factor_width
                scaled_x2 = x2/scaling_factor_width
                scaled_y1 = y1/scaling_factor_height
                scaled_y2 = y2/scaling_factor_height
                edge_tol = scaled_x2 - scaled_x1
                coords = f"{scaled_x1}, {scaled_y1}, {scaled_x2}, {scaled_y2}"
                
                tbls_scaled = camelot.read_pdf(url, flavor='stream', edge_tol=edge_tol, pages=str(page_idx+1), flag_size=True, table_areas=[coords], split_text=True) # split_text=True
                tbl_cleaned, ESG_keywords = clean_tbl(tbls_scaled[0].df)
                # there is a valid table 
                
                if tbl_cleaned is None:
                    continue
                else:
                    print("----------------------------------------------------------------------------------------------------------")
                    print("CLEANED TABLE OBTAINED!, tbl_num is {0}".format(count))
                    print("----------------------------------------------------------------------------------------------------------")
                    tbl_lst.append(tbl_cleaned)
                    keywords_lst.append(ESG_keywords)
                    # Table detection portion saving 
                    # --- start ---
                    # output_path = f"{copy_to_path}/PAGE{str(page_idx+1)}_IMAGE{str(i)}.jpg"
                    # cv2.imwrite(output_path, img)
                    # i += 1
                    # lst_imgs.append(["("+str(y)+":"+str(y+h)+", "+str(x)+":"+str(x+w)+")", output_path])
                    # --- end ---
                

        except Exception as e:
            print(e)
            print('error on page {0}'.format(page_idx+1))
            continue 
    
    # return tbl_lst, keywords_lst, lst_imgs
    return tbl_lst, keywords_lst

In [ ]:
# returns a pdf_dict => key = page, value = lst of df (tables)
# returns a keywords_dict => key=page, value = lst of lst (keywords) 
def extract_tbl_from_pdf(pdf_url, pages_to_look_for): 
    
    pdf_dict = {}
    keywords_dict = {}
    # image_path_obj = {}
    
    # consistent to all pages in a pdf_url 
    print('getting pdf & img height, width...')
    pdf_height, pdf_width = get_pdf_dimension(pdf_url) # original pdf height,width 
    img_height, img_width, pdf_pil_img = get_image_and_dimension(pdf_url) # original image height,width 
    scaling_factor_height, scaling_factor_width = get_scaling_factor(pdf_height, pdf_width, img_height, img_width)
    print('retrieved pdf & img height, width')
    
    # convert PIL images to CV images
    print('converting all pages to CV2 img...')
    pdf_cv2_img = convert_PIL_cv2(pdf_pil_img)
    print ('converted all pages to CV2 img')
    
    # pdf_cv2_img = images of all pages in a pdf 
    print('looping through each page...')
    for page_no in pages_to_look_for:
        print('retrieving from page {0}'.format(page_no))
        page_idx = int(page_no) - 1
        tbl_lst, keywords_lst = extract_tbl_from_page(page_idx, pdf_cv2_img, scaling_factor_height, scaling_factor_width, img_height, pdf_url)
        # tbl_lst, keywords_lst, img_lst = extract_tbl_from_page(page_idx, pdf_cv2_img, scaling_factor_height, scaling_factor_width, img_height, pdf_url)
        pdf_dict[page_no] = tbl_lst
        keywords_dict[page_no] = keywords_lst
        # image_path_obj[page_no] = img_lst
    
    # return pdf_dict, keywords_dict, image_path_obj
    return pdf_dict, keywords_dict

In [ ]:
def open_JSON(file_path):
    
    f = open(file_path,)
    data = json.load(f)
    
    json_lst = []
    keywords_lst = []
    count = 1
    
    # json_lst_img = []
    
    for report in data:     
        print('--------------------------NEW REPORT {0}--------------------------'.format(count))
        count += 1
        
        # basic information
        company = report['company']
        year = report['year']
        pdf_url = report['url']
        
        # create dictionary for report 
        report_dict = {}
        report_dict['company'] = company
        report_dict['year'] = year
        report_dict['pdf_url'] = pdf_url
        
        # create dictionary for keywords
        kw_dict = {}
        kw_dict['company'] = company
        kw_dict['year'] = year
        kw_dict['pdf_url'] = pdf_url
        
        # json_obj = {}
        # json_obj['company'] = company
        # json_obj['year'] = year
        # json_obj['pdf_url'] = pdf_url
        # path = output_path + company + '_' + year
        # os.mkdir(path)
        # print(f"Detection for Report: {company}_{year}")
        
        # relevant pages for ESG info extraction
        pages_to_look_for = []
        for page in report['filtered_report_tables_direct']:
            pages_to_look_for.append(page)
            
        for page in report['filtered_report_tables_indirect']:
            pages_to_look_for.append(page)
            
        # retrieve pdf via link - if link is not valid, go to next company 
        if ".pdf" not in pdf_url:
            print("{0}'s URL is not a PDF. Year is {1}".format(company, year))
            continue 
            
        else:
            # extract tbl from relevant pages in a pdf 
            try:
                print('calling extract_tbl_from_pdf')
                report_dict['tbl_pages'], kw_dict['tbl_pages'] = extract_tbl_from_pdf(pdf_url, pages_to_look_for) # returns a dict 
                # report_dict['tbl_pages'], kw_dict['tbl_pages'], json_obj['images_path']= extract_tbl_from_pdf(pdf_url, pages_to_look_for) 
                json_lst.append(report_dict)
                keywords_lst.append(kw_dict)
                print('Successfull for {0}, {1}'.format(company, year))
            except Exception as e:
                print(e)
                print("Error occurred in table_extraction/ Request failed")
                report_dict['tbl_pages'] = []
                
            # json_lst_img.append(json_obj)
    
    # return json_lst, keywords_lst, json_lst_img
    return json_lst, keywords_lst

### Data Retrieval

In [ ]:
tbl_asian_banks, keywords_asian_banks = open_JSON('/Users/aifen/Desktop/output_capstone/preprocessed/all_asian_banks_preprocessed_vfinal.json')

In [ ]:
tbl_asian_banks

In [ ]:
# saving of df 
import pickle
with open('/Users/aifen/Desktop/output_capstone/Pickle_v2/tbl_asian_banks.pickle', 'wb') as tbl_asian_banks_pickle:
    pickle.dump(tbl_asian_banks, tbl_asian_banks_pickle, protocol=pickle.HIGHEST_PROTOCOL)   

In [ ]:
# ensure it has been saved, this is just for checking 
loaded_tbl_asian_banks = pd.read_pickle(r'/Users/aifen/Desktop/output_capstone/Pickle_v2/tbl_asian_banks.pickle')

In [ ]:
# saving of keywords 
import json
with open('/Users/aifen/Desktop/output_capstone/JSON_v2/text_asian_banks.json', 'w') as fout:
    json.dump(keywords_asian_banks, fout)

In [ ]:
# saving of images - Jermaine's portion
# with open('asian_banks_output.json', 'w') as f:
#     json.dump(json_lst_img, f)